Universidad del Valle de Guatemala

Rodrigo Alvarado

16106

# Analisis de sentimiento en liricas de una cancion utilizando recurrent neural networks

## Datos
Se utiliza el dataset de canciones MoodyLyrics, proveido por Erion Çano del Politecnico de Torino. Este dataset contiene nombres de canciones y su localizacion en uno de 4 sectores correspondiendo a la escala de Valence-Arousal. Se utilizo un script de python para descargar las lyrics y posteriormente añadirlas al dataset.

In [1]:
import torch
from torchtext import data
import random

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

ID = data.Field()
AUTHOR = data.Field()
NAME = data.Field()
LYRICS = data.Field(tokenize = 'spacy')
LABEL = data.LabelField()

# fields = [('index', ID), (None, None), ('artist', AUTHOR), ('name', NAME), ('mood', LABEL), ('lyrics', LYRICS)]
fields = [(None, None), (None, None), (None, None), (None, None), ('mood', LABEL), ('lyrics', LYRICS)]

train_data, test_data = data.TabularDataset.splits(
                                        path = 'Datasets',
                                        train = 'mood_train.csv',
                                        test = 'mood_test.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [2]:
MAX_VOCAB_SIZE = 25_000

LYRICS.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)
NAME.build_vocab(train_data)

In [3]:
print(LABEL.vocab.stoi)

defaultdict(None, {'relaxed': 0, 'happy': 1, 'angry': 2, 'sad': 3})


In [4]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort_key=lambda x: len(x.lyrics),
    batch_size = BATCH_SIZE, 
    device = device)

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        # Add dropout para que no sea tan overfit
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        text = text.permute(1, 0)
        
        embedded = self.embedding(text)
        
        embedded = embedded.unsqueeze(1)
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
            
        return self.fc(cat)

In [30]:
INPUT_DIM = len(LYRICS.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4,5]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.8
PAD_IDX = LYRICS.vocab.stoi[LYRICS.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [31]:
pretrained_embeddings = LYRICS.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [ 0.4298,  0.8205, -1.4562,  ...,  1.4802,  0.2942,  1.3924],
        ...,
        [ 0.0434, -0.5822,  0.8061,  ...,  2.0718,  0.1397,  2.1618],
        [ 0.8172,  0.5621, -1.7299,  ...,  1.2331,  0.3003,  1.0739],
        [ 0.1599,  1.1768,  0.8474,  ...,  0.9024,  2.3826, -1.2036]])

In [32]:
UNK_IDX = LYRICS.vocab.stoi[LYRICS.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [33]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [34]:
def categorical_accuracy(preds, y):
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [35]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.lyrics)
        
        loss = criterion(predictions, batch.mood)
        
        acc = categorical_accuracy(predictions, batch.mood)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [36]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.lyrics)
            
            loss = criterion(predictions, batch.mood)
            
            acc = categorical_accuracy(predictions, batch.mood)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [37]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [38]:
N_EPOCHS = 50

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 1.825 | Train Acc: 26.48%
	 Val. Loss: 1.397 |  Val. Acc: 28.12%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 1.644 | Train Acc: 28.65%
	 Val. Loss: 1.354 |  Val. Acc: 28.52%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 1.521 | Train Acc: 32.29%
	 Val. Loss: 1.338 |  Val. Acc: 35.74%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 1.463 | Train Acc: 32.81%
	 Val. Loss: 1.317 |  Val. Acc: 38.67%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 1.357 | Train Acc: 36.46%
	 Val. Loss: 1.310 |  Val. Acc: 39.26%
Epoch: 06 | Epoch Time: 0m 1s
	Train Loss: 1.295 | Train Acc: 38.02%
	 Val. Loss: 1.323 |  Val. Acc: 32.81%
Epoch: 07 | Epoch Time: 0m 1s
	Train Loss: 1.266 | Train Acc: 41.93%
	 Val. Loss: 1.300 |  Val. Acc: 36.52%
Epoch: 08 | Epoch Time: 0m 1s
	Train Loss: 1.207 | Train Acc: 45.40%
	 Val. Loss: 1.277 |  Val. Acc: 44.73%
Epoch: 09 | Epoch Time: 0m 1s
	Train Loss: 1.180 | Train Acc: 48.00%
	 Val. Loss: 1.278 |  Val. Acc: 39.26%
Epoch: 10 | Epoch Time: 0m 1

In [39]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.155 | Test Acc: 47.10%


In [40]:
import spacy
nlp = spacy.load('en')

def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [LYRICS.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [41]:
pred_class = predict_class(model, "When you were here before Couldn't look you in the eye You're just like an angel Your skin makes me cry You float like a feather In a beautiful world I wish I was special You're so fuckin' special")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 1 = happy
